## CNN Feature Extraction using RestNet

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import methods 
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os

## Load the Model


In [ ]:
#Using the RestNet 50 model to extract features
model = models.resnet50(pretrained = True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()
if torch.cuda.is_available():
    model = model.cuda()

## Data PreProcessing 

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.images = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith((".jpg", ".png"))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        #Do preprocessing here 
        image_path = self.images[idx]
        rgb_image_arr = methods.convert_rgb(image_path)
        normalised_img = methods.z_normalization(rgb_image_arr)
        image = Image.fromarray(normalised_img.astype('uint8'), 'RGB')
        image_tensor = self.transform(image) if self.transform else image
        key = os.path.basename(image_path).removesuffix('.jpg').removesuffix('.png')
        return key, image_tensor

In [ ]:
original_folder_path =  '../dataverse_files/HAM10000_images_part_1'
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = ImageDataset(directory=original_folder_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=0)


## Feature Extraction

In [ ]:
#Extracting features
features_map2 = {}
with torch.no_grad():
    for batch_idx, output in enumerate(data_loader):
        if torch.cuda.is_available():
            images = images.cuda()

        
        key, images = output
        batch_features = model(images) 
        #print(batch_features.shape)
        batch_features = batch_features.view(batch_features.size(0), -1)  # Flatten features
              
        batch_features = batch_features.cpu().numpy()
        
        for i, feature in enumerate(batch_features):
            image_id = batch_idx * data_loader.batch_size + i  # Compute global image ID/index
            features_map2[key[i]] = feature
            print(f'Done for image {image_id}')


In [ ]:
#Saving the features
features_df = pd.DataFrame(features_map2)
features_df
features_df.to_json('features.json')
features_df
#np.save('features.npy', features_df)